In [ ]:
#%pip install "snowflake-connector-python>=2.6.2,<2.7"
#%pip install "snowflake-connector-python[pandas]"
#%pip install snowflake-sqlalchemy
#%pip install azure-ai-textanalytics
import snowflake.connector
import pandas as pd
import requests
import json
import time

In [109]:
# connect to snowflake database
con = snowflake.connector.connect(
    user='mmarzillo',
    password='PASSWORD',
    account='sn00102.central-us.azure',
    warehouse = 'SMALL_WH',
    database = 'TEXTANALYTICS',
)

StatementMeta(mattmarz, 12, 5, Finished, Available)

In [110]:
# query table to pandas data frame
sql = "select * from PUBLIC.MYTABLE"

cursor = con.cursor()
cursor.execute(sql)

df_notes = pd.read_sql(sql, con)
df_notes['doc_id'] = df_notes.index
df_notes.head()

StatementMeta(mattmarz, 12, 6, Finished, Available)

,PAT_ID,NOTES,doc_id
0,123456,Patient is presenting with pink eye,0
1,234567,Patient has severe back pain and needs Tylenol,1
2,345678,Patient broke her knee,2
3,456789,Patient needs a hip replacement,3
4,567890,Patient has the flu and vomits with frequency,4


In [111]:
# pull out list of text 
documents = df_notes["NOTES"].tolist()
documents

StatementMeta(mattmarz, 12, 7, Finished, Available)

['Patient is presenting with pink eye',
 'Patient has severe back pain and needs Tylenol',
 'Patient broke her knee',
 'Patient needs a hip replacement',
 'Patient has the flu and vomits with frequency',
 'Patient has a sever migraine',
 'Patient has diabetes and needs more insulin',
 'Patient has a neck injury and covid',
 'Patient fractured his toe and finger',
 'Patient has heart disease and diabetes and high blood pressure']

In [112]:
# connect to language service
key = "KEY"
endpoint = "https://matt-marz-language.cognitiveservices.azure.com/"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

StatementMeta(mattmarz, 12, 8, Finished, Available)

In [113]:
# pass docs to service
poller = client.begin_analyze_healthcare_entities(documents)
result = poller.result()

docs = [doc for doc in result if not doc.is_error]

StatementMeta(mattmarz, 12, 9, Finished, Available)

In [114]:
# reshape results into a pandas data frame
ent = []
normalized_text = []
category = []
doc_id = []
source = []
for idx, doc in enumerate(docs):
    for entity in doc.entities:
        doc_id.append(doc.id)
        ent.append(entity.text)
        normalized_text.append(entity.normalized_text)
        category.append(entity.category)
        source.append(entity.data_sources)
        
d = {'doc_id':doc_id, 'entity': ent, 'category': category, 'normalized_text': normalized_text}
df = pd.DataFrame(data = d)

StatementMeta(mattmarz, 12, 10, Finished, Available)

In [115]:
# convert data types to match for join and join data sets
df['doc_id'] = df['doc_id'].astype(int)
df_out = pd.merge(df, df_notes)
df_out

StatementMeta(mattmarz, 12, 11, Finished, Available)

,doc_id,entity,category,normalized_text,PAT_ID,NOTES
0,0,pink eye,SymptomOrSign,Conjunctivitis,123456,Patient is presenting with pink eye
1,1,severe,ConditionQualifier,None,234567,Patient has severe back pain and needs Tylenol
2,1,back pain,SymptomOrSign,Back Pain,234567,Patient has severe back pain and needs Tylenol
3,1,Tylenol,MedicationName,Tylenol,234567,Patient has severe back pain and needs Tylenol
4,2,broke her knee,SymptomOrSign,None,345678,Patient broke her knee
5,3,hip replacement,TreatmentName,Hip Replacement Arthroplasty (procedure),456789,Patient needs a hip replacement
6,4,flu,SymptomOrSign,Influenza,567890,Patient has the flu and vomits with frequency
7,4,vomits,SymptomOrSign,Vomiting,567890,Patient has the flu and vomits with frequency
8,5,sever,ConditionQualifier,None,678901,Patient has a sever migraine
9,5,migraine,Diagnosis,Migraine Disorders,678901,Patient has a sever migraine


In [135]:
# Write file to stage
currentDateTime = time.strftime('%d-%m-%Y %H:%M:%S')
df_out.to_csv('abfss://stage@STORAGENAME.dfs.core.windows.net//results'+currentDateTime+'.csv', index = False,header=False)

StatementMeta(mattmarz, 12, 31, Finished, Available)